In [1]:
##########################################
##########################################
## MAKE SURE TO ATTACH YOUR LAKEHOUSE   ##
##########################################
##########################################

StatementMeta(, c7583ca3-4e5c-4ef3-9ac6-0fd4e531d320, 3, Finished, Available)

In [2]:
# Install CAT TOOLS
%pip install "https://raw.githubusercontent.com/m-kovalsky/fabric_cat_tools/main/fabric_cat_tools-0.3.1-py3-none-any.whl" --q


StatementMeta(, c7583ca3-4e5c-4ef3-9ac6-0fd4e531d320, 9, Finished, Available)


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [3]:
##########################################
# Get Table Data
##########################################
import fabric_cat_tools as fct
import pandas as pd
import os
from pyspark.sql.functions import col, year, month, quarter, to_date, dayofmonth,current_timestamp
import time
import warnings
from pyspark.sql import SparkSession
from datetime import datetime,date,timedelta

# Get Table Details
# Reference: https://github.com/m-kovalsky/fabric_cat_tools?tab=readme-ov-file#get_lakehouse_tables

# Get the Lakehouse Table sizes into Pandas Dataframe
df_tableSizes = fct.get_lakehouse_tables(
        extended = True)

# ## Remove Invalid Characters from Column Names
df_tableSizes.columns = df_tableSizes.columns.str.replace(r'(.*\[)|(\].*)', '', regex=True)
df_tableSizes.columns = df_tableSizes.columns.str.replace(' ', '_')
df_tableSizes.columns = df_tableSizes.columns.str.replace('(', '')
df_tableSizes.columns = df_tableSizes.columns.str.replace(')', '')

spk_tableSizes = spark.createDataFrame(df_tableSizes)

# Transform data in dataframe
tables = pd.DataFrame(df_tableSizes)

## Write to Lakehouse Table
filename = 'TableSizes_' + (datetime.today()).strftime('%Y%m%d') + '.csv'
tables.to_csv("/lakehouse/default/Files/Storage_Sizes/" + filename)

#display(df_tableSizes)

StatementMeta(, c7583ca3-4e5c-4ef3-9ac6-0fd4e531d320, 11, Finished, Available)

In [4]:
##########################################
## Get the File Sizes
##########################################
import pandas as pd
import os
from pyspark.sql.functions import col, year, month, quarter, to_date, dayofmonth,current_timestamp
import time
import warnings
from pyspark.sql import SparkSession
from datetime import datetime,date,timedelta

# Reference https://stackoverflow.com/questions/61547289/loop-to-pull-file-size-folder-size-and-directory-size

# Files Location
dir_path = '/lakehouse/default/Files'

# Collect data for all files in the directory
data = {'directory': [], 'file_name': [], 'file_size': [], 'file_date': []}
for dirpath, dirnames, filenames in os.walk(dir_path):
    for f in filenames:
        filename = "{}\{}" .format(dirpath, f)
        data["directory"].append(dirpath)
        data["file_name"].append(f)
        data["file_size"].append(os.path.getsize(filename))
        data["file_date"].append(time.strftime('%m/%d/%Y', time.gmtime(os.path.getmtime(filename))))

# Transform data in dataframe
files = pd.DataFrame(data)

## Write to Lakehouse Table
filename = 'FileSizes_' + (datetime.today()).strftime('%Y%m%d') + '.csv'
files.to_csv("/lakehouse/default/Files/Storage_Sizes/" + filename)
spk_items = spark.createDataFrame(files)
# Adding Current Date Time to Lakehouse Table
# df_with_datetime = spk_items.withColumn("CurrentDateTime", current_timestamp())
# df_with_datetime.write.mode("append").format("delta").option("overwriteSchema", "true").saveAsTable('FileSizes')

display(spk_items)

StatementMeta(, c7583ca3-4e5c-4ef3-9ac6-0fd4e531d320, 12, Finished, Available)

SynapseWidget(Synapse.DataFrame, 6d2b8d65-a68f-4850-a6b3-5ba0c7eb1f77)

In [5]:
## Getting both Datasets into a Single Query
from pyspark.sql import SparkSession

# Register Spark DataFrame as a temporary table
spk_tableSizes.createOrReplaceTempView("spk_tableSizes")

# Register Spark DataFrame as a temporary table
spk_items.createOrReplaceTempView("spk_items")

# Perform a SQL query on the temporary table
query = """
Select 
   file_name as Table_Name
  ,CAST(file_size / 1024.00 / 1024.00 as DECIMAL(18,4))  as Table_Size_MB
  ,'Lakehouse Files' as FileType
  , directory 
  , current_date() as InsertedDateTime
from spk_items

UNION ALL

Select 
   Table_Name
  ,CAST(Table_Size / 1024.00 / 1024.00 as DECIMAL(18,4))  as Table_Size_MB
  ,'Lakehouse Tables' as FileType
  , 'Tables' as directory
  , current_date() as InsertedDateTime
from spk_tableSizes

"""

## Getting data into Spark DataFrame
result = spark.sql(query)
# display(file_storage)
result.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/Onelake_Storage")


StatementMeta(, c7583ca3-4e5c-4ef3-9ac6-0fd4e531d320, 13, Finished, Available)